In [39]:
from __future__ import division
from past.utils import old_div
from sippy import functionset as fset
from sippy import *
import numpy as np
import control.matlab as cnt
import matplotlib.pyplot as plt
from matplotlib import rcParams
from tf2ss import lsim

rcParams['figure.figsize'] = (9.0, 5.0)
%matplotlib nbagg

## Define sampling time and Time vector

In [40]:
sampling_time = 1.  # [s]
end_time = 400  # [s]
npts = int(old_div(end_time, sampling_time)) + 1
Time = np.linspace(0, end_time, npts)

## Define pseudo random binary sequence as input signal and white noise as noise signal

In [41]:
# make input signal
switch_probability = 0.08  # [0..1]
Usim, _, _ = fset.GBN_seq(npts, switch_probability)

# make noise for input signal
white_noise_variance = [0.005]
e_t = fset.white_noise_var(Usim.size, white_noise_variance)[0]

## Define the system

### Numerator of noise transfer function has only one root: nc = 1

In [42]:
NUM_H = [1., 0.3, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

### Common denominator between input and noise transfer functions has 4 roots: na = 4

In [43]:
DEN = [1., -2.21, 1.7494, -0.584256, 0.0684029, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

### Numerator of input transfer function has 3 roots: nb = 3

In [44]:
NUM = [1., -2.07, 1.3146]

### Define transfer functions

In [45]:
g_sample = cnt.tf(NUM, DEN, sampling_time)
h_sample = cnt.tf(NUM_H, DEN, sampling_time)
print("g()")
display(g_sample)
print("h()")
display(h_sample)

g()


TransferFunction(array([ 1.    , -2.07  ,  1.3146]), array([ 1.       , -2.21     ,  1.7494   , -0.584256 ,  0.0684029,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ]), 1.0)

h()


TransferFunction(array([1. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. ]), array([ 1.       , -2.21     ,  1.7494   , -0.584256 ,  0.0684029,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ]), 1.0)

## Time responses

### Input reponse

In [46]:
Y1, Time, Xsim = lsim(g_sample, Usim, Time)
plt.figure()
plt.plot(Time, Usim, label="u(t)")
plt.plot(Time, Y1, label="y(t)")
plt.xlabel("Time")
plt.title("Time response y(t)=g*u(t)")
plt.legend()
plt.grid()

<IPython.core.display.Javascript object>

### Noise response

In [47]:
Y2, Time, Xsim = lsim(h_sample, e_t, Time)
plt.figure()
plt.plot(Time, e_t, label="e(t)")
plt.plot(Time, Y2, label="y(t)")
plt.xlabel("Time")
plt.title("Time response y(t)=h*e(t)")
plt.legend()
plt.grid()

<IPython.core.display.Javascript object>

## Compute total output 
$$Y_t = Y_1 + Y_2 = G.u + H.e$$

In [48]:
Ytot = Y1 + Y2
Utot = Usim + e_t
plt.figure()
plt.plot(Time, Ytot, label="y_t(t)")
plt.plot(Time, Utot, label="u(t) + e(t)")
plt.xlabel("Time")
plt.title("Time response y_t(t)=g*u(t) + h*e(t)")
plt.legend()
plt.grid()

<IPython.core.display.Javascript object>

## Perform system identification from collected data

In [49]:
Id_sys = system_identification(Ytot, Usim, 'ARMAX', IC='BIC', na_ord=[2, 5], \
                               nb_ord=[1, 5], nc_ord=[0, 2], delays=[10, 13], \
                               max_iterations=1000)

Armax model:
- Params:
  na: 2 (2, 5)
  nb: 1 (1, 5)
  nc: 2 (0, 2)
  delay: 10 (10, 13)
  dt: 1.0 
  method: BIC 
  max iterations: 1000 
- Output:
  G: <TransferFunction>: sys[26]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


           -0.07533
------------------------------
z^11 - 1.664 z^10 + 0.6732 z^9

dt = 1.0
 
  H: <TransferFunction>: sys[27]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


z^11 + 0.2622 z^10 - 0.2474 z^9
-------------------------------
z^11 - 1.664 z^10 + 0.6732 z^9

dt = 1.0
 
  Vn: 0.01044872279265839 
  Model Output: [[ -1.3146      -2.6292      -3.464466    -4.01064262  -4.54464128
   -5.19205772  -5.91014519  -6.62517982  -4.64113078  -2.57443328
   -1.41525297  -0.54608541  -0.34821325   0.34120121   1.06629155
    2.05566201   3.082857     4.0723601    5.08512704   5.77545634
    1.50467469   0.82509251  -0.1897422   -0.24192628  -0.76027208
   -1.58010011  -2.53689254  -3.64156152  -4.68218953  -5.72373578
   -6.46660702  -7.28401693  -7.92048342  -8.5368

## Check that output of the identified system is consistent

In [50]:
Y_id1, Time, Xsim = lsim(Id_sys.G, Usim, Time)
Y_hid1, Time, Xsim = lsim(Id_sys.H, e_t, Time)
Y_idTot = Y_id1 + Y_hid1

In [51]:
plt.figure(0)
plt.plot(Time, Usim)
plt.ylabel("Input GBN")
plt.xlabel("Time")
plt.title("Input, validation data (Switch probability=0.08)")
plt.grid()

<IPython.core.display.Javascript object>

In [52]:
plt.figure(1)
plt.plot(Time, Ytot, label="Original system")
plt.plot(Time, Y_idTot, label="Identified system")
plt.grid()
plt.xlabel("Time")
plt.ylabel("y_tot")
plt.title("Gu+He (identification data)")
plt.legend()

<IPython.core.display.Javascript object>

In [53]:
plt.figure(2)
plt.plot(Time, Y1, label="Original system")
plt.plot(Time, Y_id1, label="Identified system")
plt.ylabel("y_out")
plt.grid()
plt.xlabel("Time")
plt.legend()
plt.title("Gu (identification data)")

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Gu (identification data)')

In [54]:
plt.figure(3)
plt.plot(Time, Y2, label="Original system")
plt.plot(Time, Y_hid1, label="Identified system")
plt.ylabel("y_err")
plt.grid()
plt.xlabel("Time")
plt.legend()
plt.title("He (identification data)")

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'He (identification data)')

# Validation of the identified system: 
## Generate new time series for input and noise

In [55]:
switch_probability = 0.07  # [0..1]
input_range = [0.5, 1.5]
U_valid, _, _ = fset.GBN_seq(npts, switch_probability, Range=input_range)
white_noise_variance = [0.01]
e_valid = fset.white_noise_var(U_valid.size, white_noise_variance)[0]

## Compute time responses for true system with new inputs

In [56]:
Yvalid1, Time, Xsim = lsim(g_sample, U_valid, Time)
Yvalid2, Time, Xsim = lsim(h_sample, e_valid, Time)
Ytotvalid = Yvalid1 + Yvalid2

## Compute time responses for identified system with new inputs

In [57]:
Yidvalid1, Time, Xsim = lsim(Id_sys.G, U_valid, Time)
Yidvalid2, Time, Xsim = lsim(Id_sys.H, e_valid, Time)
Yidtotvalid = Yidvalid1 + Yidvalid2

## Check responses are almost equal

In [58]:
plt.figure(4)
plt.plot(Time, U_valid)
plt.ylabel("Input GBN")
plt.xlabel("Time")
plt.title("Input, validation data (Switch probability=0.07)")
plt.grid()

<IPython.core.display.Javascript object>

In [59]:
plt.figure(5)
plt.plot(Time, Ytotvalid)
plt.plot(Time, Yidtotvalid)
plt.xlabel("Time")
plt.ylabel("y_tot")
plt.legend(['Original system', 'Identified system'])
plt.grid()

rmse = np.round(np.sqrt(np.mean((Ytotvalid-Yidtotvalid)**2)), 2)
plt.title("Validation: Gu+He | RMSE = {}".format(rmse))

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Validation: Gu+He | RMSE = 21.35')

In [60]:
plt.figure(6)
plt.plot(Time, Yvalid1, label="Original system")
plt.plot(Time, Yidvalid1, label="Identified system")
plt.grid()
plt.xlabel("Time")
plt.ylabel("y_out")
plt.title("Gu (Validation)")
plt.legend()

<IPython.core.display.Javascript object>

In [61]:
plt.figure(7)
plt.plot(Time, Yvalid2, label="Original system")
plt.plot(Time, Yidvalid2, label="Identified system")
plt.grid()
plt.xlabel("Time")
plt.ylabel("y_err")
plt.title("He (Validation)")
plt.legend()

<IPython.core.display.Javascript object>

In [62]:
print("Original h()")
display(h_sample)
print("Identified h()")
display(Id_sys.H)

plt.figure()
_ = cnt.frequency_response([h_sample, Id_sys.H])

Original h()


TransferFunction(array([1. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. ]), array([ 1.       , -2.21     ,  1.7494   , -0.584256 ,  0.0684029,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ]), 1.0)

Identified h()


TransferFunction(array([ 1.        ,  0.26216386, -0.24736508,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ]), array([ 1.        , -1.66359836,  0.67315849,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ]), 1.0)

<IPython.core.display.Javascript object>

In [63]:
print("Original g()")
display(g_sample)
print("Identified g()")
display(Id_sys.G)

plt.figure()
_ = cnt.frequency_response([g_sample, Id_sys.G])

Original g()


TransferFunction(array([ 1.    , -2.07  ,  1.3146]), array([ 1.       , -2.21     ,  1.7494   , -0.584256 ,  0.0684029,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ]), 1.0)

Identified g()


TransferFunction(array([-0.07532549]), array([ 1.        , -1.66359836,  0.67315849,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ]), 1.0)

<IPython.core.display.Javascript object>